In [2]:
import pandas as pd
from fuzzywuzzy import process, fuzz

mangadex_df = pd.read_csv("mangadex_optimized.csv")
mangaworld_df = pd.read_csv("mangaworld_optimized.csv")

mangadex_df.head()

manga_id  \
0  42c57bae-fc53-4aa1-b2e3-0c51e30b0521   
1  39ec041f-5bc7-46f5-a2e7-4af04fc35869   
2  3a7bd813-c6ea-4052-a6a9-6a5e617e27a6   
3  a5f502a7-9ab0-48de-9dec-c7b7bf54b905   
4  51ebd014-189e-4ff4-bac8-968ff0c2b094   

                                          title_api  \
0                            #TodaysUnderwearReport   
1  'Kore Ageru kara sa; Onee-san to Asonde kureru?'   
2                       +99 Reinforced Wooden Stick   
3                                            -Rain-   
4                                            .traeH   

                                          alt_titles          author  \
0  何故か毎日下着の色を報告してくるガールズ | nazeka mainichi shitagi...         Dojirou   
1  これあげるからさお姉さんと遊んでくれる | te daré esto jugarás con...   Nishizawa 5mm   
2  99강화나무몽둥이 | 99ganghwanamumongdungi | 99 reinfo...   Hong-Sil (홍실)   
3                   rain hayase hashiba | rain дождь  Hayase Hashiba   
4                               قلب | traeh | ецдрес    Hanatsu Yaya   

           artist demographic     status    year  \
0         Dojirou     shounen    ongoing  2020.0   
1   Nishizawa 5mm         NaN  completed  2020.0   
2    Jiperi (지페리)         NaN    ongoing  2022.0   
3  Hayase Hashiba         NaN  completed     NaN   
4    Hanatsu Yaya         NaN  completed     NaN   

                                         description content_rating  ...  \
0  Girls who really want to show off their underw...     suggestive  ...   
1  It's literally an advertisement for a figure w...     suggestive  ...   
2  Welcome to “Chrono Life;” a virtual reality ga...           safe  ...   
3  On a rainy day; when Mai got stood up by the g...        erotica  ...   
4  After being abandoned by first her father then...           safe  ...   

  kazakh  estonian  icelandic latvian czech tamil urdu telugu  \
0    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
1    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
2    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
3    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   
4    NaN       NaN        NaN     NaN   NaN   NaN  NaN    NaN   

  author_normalized artist_normalized  
0           dojirou           dojirou  
1     nishizawa 5mm     nishizawa 5mm  
2       hong-sil 홍실        jiperi 지페리  
3    hayase hashiba    hayase hashiba  
4      hanatsu yaya      hanatsu yaya  

[5 rows x 89 columns]

In [3]:
mangaworld_df.head()

title  \
0  +99 Reinforced Wooden Stick   
1                       .traeH   
2                009 Re:Cyborg   
3  1-nen C-gumi Yoshida Kakari   
4                   1/2 Prince   

                                          alt_titles          author  \
0  ['99 wooden stick', '99 ทอนไมพรอมบวก', '99 har...         Hongsil   
1                                            ['قلب']    HANATSU Yaya   
2    ['009 re cyborg', '009 re사이보그', '009 рекиборг']             NaN   
3  ['1 nen c gumi yoshida kakari', '1年c組吉田係', 'fi...  SAKURAI Shiori   
4  ['12 hoàng tử', '12 princas', '12 wangzi', '12...           Yu Wo   

            artist     type     status  year    views  \
0           Jipery   Manhwa    dropped  2022  1385162   
1     HANATSU Yaya  Oneshot  completed  2008    62678   
2              NaN    Manga  completed  2012    44944   
3   SAKURAI Shiori  Oneshot  completed  2011    46507   
4  Choi Hong Chong   Manhua  completed  2006   386591   

              title_normalized  \
0   99 reinforced wooden stick   
1                        traeh   
2                 009 recyborg   
3  1-nen c-gumi yoshida kakari   
4                    12 prince   

                              alt_titles_by_language  ... albanian czech  \
0  {'slovak': ["'99 wooden stick'"], 'thai': ["'9...  ...      NaN   NaN   
1                              {'arabic': ["'قلب'"]}  ...      NaN   NaN   
2  {'danish': ["'009 re cyborg'"], 'korean': ["'0...  ...      NaN   NaN   
3  {'swahili': ["'1 nen c gumi yoshida kakari'", ...  ...      NaN   NaN   
4  {'vietnamese': ["'12 hoàng tử'"], 'italian': [...  ...      NaN   NaN   

  kazakh icelandic latvian azerbaijani urdu matched_title_normalized  \
0    NaN       NaN     NaN         NaN  NaN                      NaN   
1    NaN       NaN     NaN         NaN  NaN                      NaN   
2    NaN       NaN     NaN         NaN  NaN                      NaN   
3    NaN       NaN     NaN         NaN  NaN                      NaN   
4    NaN       NaN     NaN         NaN  NaN                      NaN   

  author_normalized artist_normalized  
0           hongsil            jipery  
1      hanatsu yaya      hanatsu yaya  
2               NaN               NaN  
3    sakurai shiori    sakurai shiori  
4             yu wo   choi hong chong  

[5 rows x 87 columns]

In [4]:
language_columns = [col for col in mangadex_df.columns if col not in [
    "manga_id", "title_api", "alt_titles", "author", "artist", "demographic",
    "status", "year", "description", "content_rating", "last_updated",
    "follows", "rating", "title_normalized", "alt_titles_by_language",
    "author_normalized", "artist_normalized"
]]

author_artist_columns = ["author_normalized", "artist_normalized"]

In [5]:
def match_main_titles(row, mangadex_titles):
    match = process.extractOne(row["title_normalized"], mangadex_titles, scorer=fuzz.ratio)
    return match[0] if match and match[1] > 85 else None

mangadex_titles = mangadex_df["title_normalized"].dropna().tolist()
mangaworld_df["matched_title_normalized"] = mangaworld_df.apply(match_main_titles, args=(mangadex_titles,), axis=1)

In [6]:
mangaworld_df["matched_title_normalized"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2906 entries, 0 to 2905
Series name: matched_title_normalized
Non-Null Count  Dtype 
--------------  ----- 
1283 non-null   object
dtypes: object(1)
memory usage: 22.8+ KB


In [7]:
mangadex_titles = mangadex_df["title_normalized"].dropna().tolist()

language_columns = set(mangadex_df.columns) & set(mangaworld_df.columns) - {
    "title_normalized", "alt_titles", "author", "artist", 
    "author_normalized", "artist_normalized", "alt_titles_by_language", 
    "matched_title_normalized"
}

non_matched_df = mangaworld_df[mangaworld_df["matched_title_normalized"].isna()].copy()

def match_title_with_confirmation(row, mangadex_titles, language_columns):
    title = str(row["title_normalized"]).strip().lower()
    match = process.extractOne(title, mangadex_titles, scorer=fuzz.ratio)

    
    if match and match[1] > 75:
        possible_match = match[0]

        for col in language_columns:
            mw_values = row[col]

            if pd.notna(mw_values):
                mw_titles = [str(t).strip().lower() for t in str(mw_values).strip("[]").split(",") if str(t).strip()]
                
                md_titles = mangadex_df.set_index("title_normalized")[col].dropna().explode().reset_index()
                lang_dict = {str(r[col]).strip().lower(): r["title_normalized"] for _, r in md_titles.iterrows()}

                for mw_title in mw_titles:
                    if mw_title in lang_dict and lang_dict[mw_title] == possible_match:
                        return possible_match

                    lang_match = process.extractOne(mw_title, lang_dict.keys(), scorer=fuzz.ratio)
                    if lang_match and lang_match[1] > 85 and lang_dict[lang_match[0]] == possible_match:
                        return possible_match
    
    return None



In [8]:

non_matched_df["matched_title_normalized"] = non_matched_df.apply(
    match_title_with_confirmation, args=(mangadex_titles, language_columns), axis=1
)
mangaworld_df.loc[non_matched_df.index, "matched_title_normalized"] = non_matched_df["matched_title_normalized"]

print("Totale matched_title_normalized dopo fase 2:", mangaworld_df["matched_title_normalized"].notna().sum())

mangaworld_df["matched_title_normalized"].info()


Totale matched_title_normalized dopo fase 2: 1317
<class 'pandas.core.series.Series'>
RangeIndex: 2906 entries, 0 to 2905
Series name: matched_title_normalized
Non-Null Count  Dtype 
--------------  ----- 
1317 non-null   object
dtypes: object(1)
memory usage: 22.8+ KB


In [9]:
mangaworld_df["matched_title_normalized"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2906 entries, 0 to 2905
Series name: matched_title_normalized
Non-Null Count  Dtype 
--------------  ----- 
1317 non-null   object
dtypes: object(1)
memory usage: 22.8+ KB


In [10]:

title_to_id_dict = pd.Series(mangadex_df["manga_id"].values, index=mangadex_df["title_normalized"]).to_dict()

linguistic_columns = [col for col in mangadex_df.columns if col not in [
    "manga_id", "title_api", "alt_titles", "author", "artist", "demographic",
    "status", "year", "description", "content_rating", "last_updated",
    "follows", "rating", "alt_titles_by_language",
    "author_normalized", "artist_normalized", "matched_title_normalized"
]]

for col in linguistic_columns:
    exploded_titles = mangadex_df.set_index("manga_id")[col].dropna().explode().reset_index()
    for _, row in exploded_titles.iterrows():
        title_to_id_dict[row[col].strip()] = row["manga_id"]



mangaworld_df["manga_id"] = mangaworld_df["matched_title_normalized"].map(title_to_id_dict)
print("MangaWorld ID unici dopo il mapping:", mangaworld_df["manga_id"].nunique())
mangaworld_df.head(10)

mangaworld_df.to_csv("mangaworld_matched.csv", index=False)

MangaWorld ID unici dopo il mapping: 1289
